In [150]:
import keras
from keras.layers.core import Activation, Dropout, Dense, Flatten
from keras.layers.recurrent import LSTM
import numpy as np
import pickle

In [151]:
class Fonts:
    def __init__(self, name, d):
        self.name = name
        self.d = d

f = open("savedFonts", 'r')
fonts = pickle.load(f)

In [153]:
#load commands
commands = []
for i, font in enumerate(fonts):
    commands.append([])
    for command in font.d:
        commands[i].append(command[0])

print commands

[['M', 'q', 't', 't', 't', 'q', 't', 't', 't', 'z', 'M', 'v', 'q', 't', 't', 'h', 'v', 'h', 'v', 'h', 'q', 't', 't', 'v', 'q', 't', 't', 'h', 'v', 'q', 'h', 'z'], ['M', 'l', 'h', 'v', 'h', 'v', 'z', 'M', 'q', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 'z'], ['M', 'q', 't', 'l', 'q', 'q', 'q', 't', 'v', 'q', 't', 'q', 'v', 'q', 't', 'q', 't', 'q', 't', 't', 't', 'q', 't', 'q', 't', 'h', 'q', 't', 'q', 't', 't', 't', 'q', 't', 't', 't', 'q', 't', 't', 't', 'v', 'q', 't', 'v', 'l', 'v', 'q', 't', 'v', 'q', 't', 't', 't', 'q', 't', 'q', 't', 'q', 'q', 'v', 'q', 't', 't', 't', 'z', 'M', 'q', 't', 't', 't', 'q', 't', 'q', 't', 'q', 't', 'q', 't', 't', 't', 'q', 't', 'q', 't', 'q', 't', 'z'], ['M', 'q', 'v', 'h', 'v', 'q', 'v', 'q', 'v', 'h', 'v', 'z', 'M', 'q', 't', 'q', 't', 't', 't', 'q', 't', 't', 't', 'z'], ['M', 'q', 't', 'q', 't', 'q', 't', 'q', 't', 'z', 'M', 'l', 'h', 'v', 'h', 'v', 'z'], ['M', 'q', 'q', 't', 'q', 't', 'z', 'M', 'q', 'q', 'q', 'l', 'q'

In [154]:
#Append '*' x stepSize in front of every command
stepSize = 4
for index in range(len(commands)):
    commands[index] = ['*' for i in range(stepSize)] + commands[index]

print commands

[['*', '*', '*', '*', 'M', 'q', 't', 't', 't', 'q', 't', 't', 't', 'z', 'M', 'v', 'q', 't', 't', 'h', 'v', 'h', 'v', 'h', 'q', 't', 't', 'v', 'q', 't', 't', 'h', 'v', 'q', 'h', 'z'], ['*', '*', '*', '*', 'M', 'l', 'h', 'v', 'h', 'v', 'z', 'M', 'q', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 'z'], ['*', '*', '*', '*', 'M', 'q', 't', 'l', 'q', 'q', 'q', 't', 'v', 'q', 't', 'q', 'v', 'q', 't', 'q', 't', 'q', 't', 't', 't', 'q', 't', 'q', 't', 'h', 'q', 't', 'q', 't', 't', 't', 'q', 't', 't', 't', 'q', 't', 't', 't', 'v', 'q', 't', 'v', 'l', 'v', 'q', 't', 'v', 'q', 't', 't', 't', 'q', 't', 'q', 't', 'q', 'q', 'v', 'q', 't', 't', 't', 'z', 'M', 'q', 't', 't', 't', 'q', 't', 'q', 't', 'q', 't', 'q', 't', 't', 't', 'q', 't', 'q', 't', 'q', 't', 'z'], ['*', '*', '*', '*', 'M', 'q', 'v', 'h', 'v', 'q', 'v', 'q', 'v', 'h', 'v', 'z', 'M', 'q', 't', 'q', 't', 't', 't', 'q', 't', 't', 't', 'z'], ['*', '*', '*', '*', 'M', 'q', 't', 'q', 't', 'q', 't', 'q', 't', 'z', 

In [155]:
#Remove long commands
maxCommandLen = 50
count = 0
for command in commands:
    command.append('.')
    if len(command) > maxCommandLen:
        commands.remove(command)
        count += 1
        
print 'Removed: ', count
print 'Max Len', max([len(command) for command in commands])

Removed:  8
Max Len 47


In [156]:
#There are no other upper cased commands
for command in commands:
    for c in command:
        if c != 'M' and c.isupper():
            print c

In [157]:
#Create dictionary for mapping to one hot vectors: oneHotMap gives letter to index
commandList = ['*','M','m','l','h','v','z','c','s','q','t','a','.']
oneHotMap = {}
for i, c in enumerate(commandList):
    oneHotMap[c] = i

In [158]:
print oneHotMap

{'a': 11, 'c': 7, 'z': 6, 'M': 1, '*': 0, 'm': 2, 'l': 3, '.': 12, 'q': 9, 's': 8, 't': 10, 'v': 5, 'h': 4}


In [159]:
#ont hot commands
oneHotCommands = np.zeros((len(commands), maxCommandLen, len(commandList)))

noOfFeatures = len(commandList)
noOfCommands = len(oneHotCommands)

In [160]:
#populate X train
for i, command in enumerate(commands):
    for j, c in enumerate(command):
        oneHotCommands[i, j, oneHotMap[c]] = 1
        
print oneHotCommands

[[[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [161]:
X = np.zeros((noOfCommands, maxCommandLen, stepSize, noOfFeatures))
Y = np.zeros((noOfCommands, maxCommandLen, noOfFeatures))

print X.shape, Y.shape

(113, 50, 4, 13) (113, 50, 13)


In [162]:
for i, command in enumerate(commands):
    for j in range(len(command) - stepSize):
        X[i,j] = oneHotCommands[i,j:j+stepSize]
        Y[i,j] = oneHotCommands[i,j+stepSize]

In [163]:
# model = keras.Sequential()

# model.add(LSTM(input_shape = (3,noOfFeatures), output_dim = 50))
# model.add(LSTM(output_dim = 100))
# model.add(Dense(output_dim = noOfFeatures))
# model.add(Activation('linear'))
# model.add(loss = 'mse', optimizer = 'rmsprop')

In [164]:
print X[0], ">>\n", Y[0]

[[[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]] >>
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0.

In [165]:
model = keras.Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(stepSize,  noOfFeatures)))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(noOfFeatures))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_40 (LSTM)               (None, 4, 64)             19968     
_________________________________________________________________
lstm_41 (LSTM)               (None, 4, 64)             33024     
_________________________________________________________________
lstm_42 (LSTM)               (None, 4, 64)             33024     
_________________________________________________________________
flatten_4 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 13)                3341      
_________________________________________________________________
activation_8 (Activation)    (None, 13)                0         
Total params: 89,357
Trainable params: 89,357
Non-trainable params: 0
_________________________________________________________________


In [166]:
for idx in range(len(X)):
    hist = model.fit(X[idx], Y[idx], batch_size=maxCommandLen, nb_epoch=15, verbose=2)

Epoch 1/15
 - 3s - loss: 1.6926
Epoch 2/15
 - 0s - loss: 1.6825
Epoch 3/15
 - 0s - loss: 1.6716
Epoch 4/15
 - 0s - loss: 1.6562
Epoch 5/15
 - 0s - loss: 1.6357
Epoch 6/15
 - 0s - loss: 1.6070
Epoch 7/15
 - 0s - loss: 1.5675
Epoch 8/15
 - 0s - loss: 1.5135
Epoch 9/15
 - 0s - loss: 1.4463
Epoch 10/15
 - 0s - loss: 1.3706
Epoch 11/15
 - 0s - loss: 1.2982
Epoch 12/15
 - 0s - loss: 1.2451
Epoch 13/15
 - 0s - loss: 1.2128
Epoch 14/15
 - 0s - loss: 1.1953
Epoch 15/15
 - 0s - loss: 1.1830
Epoch 1/15
 - 0s - loss: 0.8989
Epoch 2/15
 - 0s - loss: 0.8385
Epoch 3/15
 - 0s - loss: 0.8029
Epoch 4/15
 - 0s - loss: 0.7787
Epoch 5/15
 - 0s - loss: 0.7603
Epoch 6/15
 - 0s - loss: 0.7454
Epoch 7/15
 - 0s - loss: 0.7329
Epoch 8/15
 - 0s - loss: 0.7220
Epoch 9/15
 - 0s - loss: 0.7125
Epoch 10/15
 - 0s - loss: 0.7040
Epoch 11/15
 - 0s - loss: 0.6963
Epoch 12/15
 - 0s - loss: 0.6893
Epoch 13/15
 - 0s - loss: 0.6828
Epoch 14/15
 - 0s - loss: 0.6769
Epoch 15/15
 - 0s - loss: 0.6715
Epoch 1/15
 - 0s - loss: 0.9

Epoch 14/15
 - 0s - loss: 0.2670
Epoch 15/15
 - 0s - loss: 0.2632
Epoch 1/15
 - 0s - loss: 1.9702
Epoch 2/15
 - 0s - loss: 1.5238
Epoch 3/15
 - 0s - loss: 1.3529
Epoch 4/15
 - 0s - loss: 1.2689
Epoch 5/15
 - 0s - loss: 1.2095
Epoch 6/15
 - 0s - loss: 1.1645
Epoch 7/15
 - 0s - loss: 1.1294
Epoch 8/15
 - 0s - loss: 1.1021
Epoch 9/15
 - 0s - loss: 1.0808
Epoch 10/15
 - 0s - loss: 1.0642
Epoch 11/15
 - 0s - loss: 1.0502
Epoch 12/15
 - 0s - loss: 1.0378
Epoch 13/15
 - 0s - loss: 1.0266
Epoch 14/15
 - 0s - loss: 1.0162
Epoch 15/15
 - 0s - loss: 1.0064
Epoch 1/15
 - 0s - loss: 0.7533
Epoch 2/15
 - 0s - loss: 0.7014
Epoch 3/15
 - 0s - loss: 0.6694
Epoch 4/15
 - 0s - loss: 0.6498
Epoch 5/15
 - 0s - loss: 0.6364
Epoch 6/15
 - 0s - loss: 0.6263
Epoch 7/15
 - 0s - loss: 0.6178
Epoch 8/15
 - 0s - loss: 0.6104
Epoch 9/15
 - 0s - loss: 0.6038
Epoch 10/15
 - 0s - loss: 0.5976
Epoch 11/15
 - 0s - loss: 0.5918
Epoch 12/15
 - 0s - loss: 0.5863
Epoch 13/15
 - 0s - loss: 0.5811
Epoch 14/15
 - 0s - loss: 0.

Epoch 12/15
 - 0s - loss: 0.7717
Epoch 13/15
 - 0s - loss: 0.7334
Epoch 14/15
 - 0s - loss: 0.7035
Epoch 15/15
 - 0s - loss: 0.6787
Epoch 1/15
 - 0s - loss: 2.3361
Epoch 2/15
 - 0s - loss: 1.8912
Epoch 3/15
 - 0s - loss: 1.6074
Epoch 4/15
 - 0s - loss: 1.4044
Epoch 5/15
 - 0s - loss: 1.2548
Epoch 6/15
 - 0s - loss: 1.1402
Epoch 7/15
 - 0s - loss: 1.0500
Epoch 8/15
 - 0s - loss: 0.9777
Epoch 9/15
 - 0s - loss: 0.9183
Epoch 10/15
 - 0s - loss: 0.8686
Epoch 11/15
 - 0s - loss: 0.8275
Epoch 12/15
 - 0s - loss: 0.7926
Epoch 13/15
 - 0s - loss: 0.7632
Epoch 14/15
 - 0s - loss: 0.7376
Epoch 15/15
 - 0s - loss: 0.7154
Epoch 1/15
 - 0s - loss: 0.7952
Epoch 2/15
 - 0s - loss: 0.6844
Epoch 3/15
 - 0s - loss: 0.6094
Epoch 4/15
 - 0s - loss: 0.5455
Epoch 5/15
 - 0s - loss: 0.4830
Epoch 6/15
 - 0s - loss: 0.4218
Epoch 7/15
 - 0s - loss: 0.3690
Epoch 8/15
 - 0s - loss: 0.3297
Epoch 9/15
 - 0s - loss: 0.3031
Epoch 10/15
 - 0s - loss: 0.2853
Epoch 11/15
 - 0s - loss: 0.2723
Epoch 12/15
 - 0s - loss: 0.

Epoch 10/15
 - 0s - loss: 0.5566
Epoch 11/15
 - 0s - loss: 0.5432
Epoch 12/15
 - 0s - loss: 0.5311
Epoch 13/15
 - 0s - loss: 0.5212
Epoch 14/15
 - 0s - loss: 0.5134
Epoch 15/15
 - 0s - loss: 0.5119
Epoch 1/15
 - 0s - loss: 0.3112
Epoch 2/15
 - 0s - loss: 0.1523
Epoch 3/15
 - 0s - loss: 0.1190
Epoch 4/15
 - 0s - loss: 0.1028
Epoch 5/15
 - 0s - loss: 0.0917
Epoch 6/15
 - 0s - loss: 0.0832
Epoch 7/15
 - 0s - loss: 0.0764
Epoch 8/15
 - 0s - loss: 0.0707
Epoch 9/15
 - 0s - loss: 0.0660
Epoch 10/15
 - 0s - loss: 0.0621
Epoch 11/15
 - 0s - loss: 0.0588
Epoch 12/15
 - 0s - loss: 0.0560
Epoch 13/15
 - 0s - loss: 0.0536
Epoch 14/15
 - 0s - loss: 0.0516
Epoch 15/15
 - 0s - loss: 0.0498
Epoch 1/15
 - 0s - loss: 0.0483
Epoch 2/15
 - 0s - loss: 0.0469
Epoch 3/15
 - 0s - loss: 0.0457
Epoch 4/15
 - 0s - loss: 0.0446
Epoch 5/15
 - 0s - loss: 0.0437
Epoch 6/15
 - 0s - loss: 0.0428
Epoch 7/15
 - 0s - loss: 0.0420
Epoch 8/15
 - 0s - loss: 0.0412
Epoch 9/15
 - 0s - loss: 0.0406
Epoch 10/15
 - 0s - loss: 0.

Epoch 8/15
 - 0s - loss: 0.1392
Epoch 9/15
 - 0s - loss: 0.1301
Epoch 10/15
 - 0s - loss: 0.1224
Epoch 11/15
 - 0s - loss: 0.1157
Epoch 12/15
 - 0s - loss: 0.1098
Epoch 13/15
 - 0s - loss: 0.1048
Epoch 14/15
 - 0s - loss: 0.1009
Epoch 15/15
 - 0s - loss: 0.0976
Epoch 1/15
 - 0s - loss: 0.1299
Epoch 2/15
 - 0s - loss: 0.0771
Epoch 3/15
 - 0s - loss: 0.0623
Epoch 4/15
 - 0s - loss: 0.0544
Epoch 5/15
 - 0s - loss: 0.0492
Epoch 6/15
 - 0s - loss: 0.0457
Epoch 7/15
 - 0s - loss: 0.0432
Epoch 8/15
 - 0s - loss: 0.0413
Epoch 9/15
 - 0s - loss: 0.0398
Epoch 10/15
 - 0s - loss: 0.0386
Epoch 11/15
 - 0s - loss: 0.0376
Epoch 12/15
 - 0s - loss: 0.0368
Epoch 13/15
 - 0s - loss: 0.0361
Epoch 14/15
 - 0s - loss: 0.0355
Epoch 15/15
 - 0s - loss: 0.0349
Epoch 1/15
 - 0s - loss: 0.5065
Epoch 2/15
 - 0s - loss: 0.3399
Epoch 3/15
 - 0s - loss: 0.2403
Epoch 4/15
 - 0s - loss: 0.1898
Epoch 5/15
 - 0s - loss: 0.1643
Epoch 6/15
 - 0s - loss: 0.1461
Epoch 7/15
 - 0s - loss: 0.1318
Epoch 8/15
 - 0s - loss: 0.1

Epoch 6/15
 - 0s - loss: 0.5928
Epoch 7/15
 - 0s - loss: 0.5275
Epoch 8/15
 - 0s - loss: 0.4739
Epoch 9/15
 - 0s - loss: 0.4288
Epoch 10/15
 - 0s - loss: 0.3932
Epoch 11/15
 - 0s - loss: 0.3665
Epoch 12/15
 - 0s - loss: 0.3456
Epoch 13/15
 - 0s - loss: 0.3284
Epoch 14/15
 - 0s - loss: 0.3136
Epoch 15/15
 - 0s - loss: 0.3005
Epoch 1/15
 - 0s - loss: 2.1751
Epoch 2/15
 - 0s - loss: 1.1140
Epoch 3/15
 - 0s - loss: 0.6954
Epoch 4/15
 - 0s - loss: 0.5753
Epoch 5/15
 - 0s - loss: 0.5363
Epoch 6/15
 - 0s - loss: 0.5139
Epoch 7/15
 - 0s - loss: 0.4975
Epoch 8/15
 - 0s - loss: 0.4843
Epoch 9/15
 - 0s - loss: 0.4732
Epoch 10/15
 - 0s - loss: 0.4634
Epoch 11/15
 - 0s - loss: 0.4547
Epoch 12/15
 - 0s - loss: 0.4468
Epoch 13/15
 - 0s - loss: 0.4394
Epoch 14/15
 - 0s - loss: 0.4324
Epoch 15/15
 - 0s - loss: 0.4258
Epoch 1/15
 - 0s - loss: 1.3416
Epoch 2/15
 - 0s - loss: 0.6880
Epoch 3/15
 - 0s - loss: 0.4459
Epoch 4/15
 - 0s - loss: 0.2994
Epoch 5/15
 - 0s - loss: 0.2176
Epoch 6/15
 - 0s - loss: 0.1

Epoch 4/15
 - 0s - loss: 0.2921
Epoch 5/15
 - 0s - loss: 0.2293
Epoch 6/15
 - 0s - loss: 0.1881
Epoch 7/15
 - 0s - loss: 0.1551
Epoch 8/15
 - 0s - loss: 0.1397
Epoch 9/15
 - 0s - loss: 0.1340
Epoch 10/15
 - 0s - loss: 0.1261
Epoch 11/15
 - 0s - loss: 0.1219
Epoch 12/15
 - 0s - loss: 0.1131
Epoch 13/15
 - 0s - loss: 0.1092
Epoch 14/15
 - 0s - loss: 0.1060
Epoch 15/15
 - 0s - loss: 0.1073
Epoch 1/15
 - 0s - loss: 0.1204
Epoch 2/15
 - 0s - loss: 0.0642
Epoch 3/15
 - 0s - loss: 0.0467
Epoch 4/15
 - 0s - loss: 0.0402
Epoch 5/15
 - 0s - loss: 0.0370
Epoch 6/15
 - 0s - loss: 0.0352
Epoch 7/15
 - 0s - loss: 0.0341
Epoch 8/15
 - 0s - loss: 0.0333
Epoch 9/15
 - 0s - loss: 0.0327
Epoch 10/15
 - 0s - loss: 0.0323
Epoch 11/15
 - 0s - loss: 0.0319
Epoch 12/15
 - 0s - loss: 0.0316
Epoch 13/15
 - 0s - loss: 0.0313
Epoch 14/15
 - 0s - loss: 0.0310
Epoch 15/15
 - 0s - loss: 0.0308
Epoch 1/15
 - 0s - loss: 0.1467
Epoch 2/15
 - 0s - loss: 0.0916
Epoch 3/15
 - 0s - loss: 0.0587
Epoch 4/15
 - 0s - loss: 0.0

In [125]:
# #Roll test
# inp[0,3,1] = 1
# inp[0,3,0] = 0
# print inp
# print np.roll(inp, -1, axis=1)

[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


In [167]:
inp = np.zeros((1,stepSize, noOfFeatures))
inp[:stepSize] = X[0,0,:stepSize]
print inp.shape

(1, 4, 13)


In [168]:
image = np.zeros((maxCommandLen,noOfFeatures))
for i in range(maxCommandLen):
    prob = model.predict_proba(inp)
    index = np.argmax(prob)
    print inp
    inp = np.roll(inp, -1, axis=1)
    inp[0,stepSize-1] = np.zeros((noOfFeatures))
    inp[0,stepSize-1,index] = 1
    image[i] = inp[0,stepSize-1]

[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]]
[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]]
[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]]
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 

In [170]:
print image

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.